# **Pipeline for Training and Testing**

## **Dependencies Installation**
Install the necessary libraries:

In [ ]:
!pip install ultralytics

## **Import Packages and Define Variables**

In [ ]:
import os
import random
import torch
from ultralytics import YOLO

TRAIN = 0 # 1 for training, 0 for inference 
MODEL_PATH = "yolo11n.pt" if TRAIN else "runs/detect/train/weights/best.pt" # path to trained model
DATA_PATH = "archive/data.yaml" # path to dataset yaml file
# dataset is available on Dropbox: https://www.dropbox.com/scl/fi/xpx2lxfpssbzrpkh584dw/archive.zip?rlkey=5a8msacr31h5nbl4gtfv0kxih&st=ksy4jwgq&dl=0

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPOCHS = 20
BATCH_SIZE = 64
IMAGE_SIZE = 640

## **Define Model and Train**
1. **Model Initialization**:
   - The `YOLO(MODEL_PATH)` initializes the YOLO model with the specified weights (e.g., `best.pt`). This weight file contains pre-trained parameters to kick-start the training process.

2. **Training Process**:
   - The `model.train()` method begins the training process using the configuration provided (dataset path, epochs, batch size, image size, device, augmentation settings, etc.).
   - Augmentation techniques such as flipping, HSV adjustments, mosaic augmentation, and mixup are applied to enhance the dataset's variability and improve generalization.

In [ ]:
model = YOLO(MODEL_PATH)

# Train the model
if TRAIN:
    results = model.train(
        data=DATA_PATH,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMAGE_SIZE,
        device=DEVICE,
        augment=True,
        flipud=0.5,    # Probability for vertical flip
        fliplr=0.5,    # Probability for horizontal flip
        hsv_h=0.015,    # Adjust hue
        hsv_s=0.7,     # Adjust saturation
        hsv_v=0.4,     # Adjust value
    )

3. **Results Object**:
   - After training, the `results` object contains key metrics and logs, such as loss values and training progress. Access the values using:

In [ ]:
if TRAIN:
    for k,v in results.results_dict.items():
        print(f'{k}: {v}')

4. **Folder Structure**:
   - During the training, `model.train()` creates a hierarchical folder structure inside the `runs/detect` directory. The structure typically includes:
     - **`train/`**: A subfolder for the training run.
       - **Weights (`weights/`)**: Contains the `best.pt` and `last.pt` weight files:
         - `best.pt`: The model weights with the best performance based on validation metrics.
         - `last.pt`: The weights from the most recent epoch.
       - **Logs**: Includes details of training metrics, such as loss values, mAP, precision, and recall.
       - **Results**: Visualization outputs like plots showing training metrics across epochs.
       - **Other Subfolders**: Depending on additional configurations (e.g., saved validation results or predictions).

5. **Purpose**:
   - The `runs` directory serves as a structured repository for all training-related artifacts, allowing to easily access trained weights, monitor training metrics, and debug issues.

## **Validation and Testing**
- **Validation:** The validation dataset (`../valid/images`) is used to monitor performance metrics like precision, recall, and mAP (mean Average Precision).

In [ ]:
# Inference

if TRAIN:
    results = model.val(
        data=DATA_PATH,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMAGE_SIZE,
        device=DEVICE
    )

- **Testing:** After training, evaluate the model on the test dataset (`../test/images`) and visualise the images. Only a limited subset of the test data is utilised within this repository; the remaining data is stored on Dropbox.

In [ ]:
dirname = "test"
filenames = os.listdir(dirname)
random.shuffle(filenames)

for filename in filenames:
    path = os.path.join(dirname, filename)
    results = model.predict(path)
    for result in results:
        result.show()

## **Exporting**
- Run `python3 exporter.py` in terminal to export the model as `.engine` file in order to use it in the `detector.py` node.

### Further Adjustments
- **Adjust Learning Rate:** Experiment with a different initial learning rate (e.g., `lr0=0.001`) to improve convergence and avoid overshooting optimal weights. But first, define a custom optimizer since YOLO ignores the entry when 'optimizer=auto'.
- **Class-Specific Training:** Use the `classes` parameter to focus training on specific classes that are underperforming or require prioritization.
- **Fine-Tune Augmentation Probabilities:** Adjust the probabilities to test the impact on generalization.
- **Hue, Saturation, and Value Adjustment:** Experiment with subtle changes to improve color transformations.

[Link](https://docs.ultralytics.com/usage/cfg/) for further configuration: